## GARCH (Generalized Autoregressive Conditional Heteroskedasticity) Model 

It is the generalized version of ARCH method. This generalization is expressed in including past variances as well as past squared residuals to estimate current (and subsequent) variances. The generalization comes from the fact that including a single past variance would (in theory) contain in itself the explanatory power of all other previous squared error terms. It serves as a sort of ARMA equivalent to the ARCH, where we’re including both past values and past errors (albeit squared).


$Var(y_t | y_{t-1}) = \omega + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}$

- $Var(y_t | y_{t-1})$: The variance today is conditional on the values of the variable yesterday. 
- $\omega$: Constant 
- $\alpha_1$: Numeric coefficient for the squared residuals for the past period. 
- $\epsilon^2_{t-1}$: Squared residuals for the past period. 
- $\beta_1$: Numeric coefficient for the squared residuals for the last period. 
- $\sigma^2_{t-1}$: Conditional variance from last period. **Volatility clustering**, or a benchmark on what to expect. The volatility is continuous and doesn't jump, and drop.


#### GARCH Vs. ARMA

- GARCH(1,1): First 1 -> Past $\epsilon^2_t$, second 1 -> Past $\sigma^2_t$ 
- ARMA(1,1) Past $y_t$, second 1 -> Past $\epsilon_t$


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from scipy.stats.distributions import chi2
import statsmodels.tsa.stattools as sts
import seaborn as sns
sns.set()


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
raw_csv_data = pd.read_csv("./../datasets/Index2018.csv")
df = raw_csv_data.copy()
df.date = pd.to_datetime(df.date, dayfirst=True)
df.set_index("date", inplace=True)
df = df.asfreq('b')
df = df.fillna(method='ffill')


In [4]:
df.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


In [5]:
# picking market value for FTSE
df['market_value'] = df.ftse

# df.drop(["ftse", "nikkei", "dax"], axis=1, inplace=True)
df.describe()


,spx,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6277.000000,6277.000000
mean,1288.642547,6083.381061,5423.679824,14597.672753,5423.679824
std,487.868210,2755.563853,1145.616719,4043.795272,1145.616719
min,438.920000,1911.700000,2876.600000,7054.980000,2876.600000
25%,992.715221,4070.460000,4486.730000,10701.130000,4486.730000
50%,1233.761241,5774.260000,5663.300000,15030.510000,5663.300000
75%,1460.250000,7445.560000,6304.630175,17860.470000,6304.630175
max,2872.867839,13559.600000,7778.637689,24124.150000,7778.637689


In [6]:
train_locs = int(df.shape[0]*0.8)
df, df_test = df.iloc[:train_locs], df.iloc[train_locs:]
df.shape, df_test.shape

((5021, 5), (1256, 5))

In [7]:
def llr_test(model_one, model_two, df=1):
    l1 = model_one.fit().llf
    l2 = model_two.fit().llf
    lr = (2*(l2-l1))
    p = chi2.sf(lr, df).round(3)
    return p

Creating returns 

In [9]:
df['returns'] = df.market_value.pct_change(1) * 100 
df.returns[:5]

date
1994-01-07         NaN
1994-01-10   -0.156704
1994-01-11   -0.779229
1994-01-12   -1.222988
1994-01-13   -0.356166
Freq: B, Name: returns, dtype: float64

The simple GARCH Model

We will fit the serially uncorrelated data - mean model does not rely on past values and errors. Basically, it is just a constant mean model.

In [10]:
model_garch_1_1 = arch_model(df.returns[1:], mean='COnstant', vol='GARCH', p=1, q=1)
result_garch_1_1 = model_garch_1_1.fit() #param, update_freq = 5
result_garch_1_1.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 6579303469.390623
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2701100877.2298183
Iteration:      3,   Func. Count:     23,   Neg. LLF: 7009.030632045198
Iteration:      4,   Func. Count:     29,   Neg. LLF: 7024.035884053223
Iteration:      5,   Func. Count:     35,   Neg. LLF: 7010.712866814991
Iteration:      6,   Func. Count:     41,   Neg. LLF: 6975.418107495356
Iteration:      7,   Func. Count:     47,   Neg. LLF: 7092.271289251072
Iteration:      8,   Func. Count:     53,   Neg. LLF: 6973.8792679196495
Iteration:      9,   Func. Count:     59,   Neg. LLF: 6970.088049064454
Iteration:     10,   Func. Count:     64,   Neg. LLF: 6970.058478417896
Iteration:     11,   Func. Count:     69,   Neg. LLF: 6970.058367475591
Iteration:     12,   Func. Count:     74,   Neg. LLF: 6970.058366189888
Iteration:     13,   Func. Count:     78,   Neg. LLF: 6970.058366189172
Optimization terminated successfully    (Exit mode 0)
        

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13948.1
Method:            Maximum Likelihood   BIC:                           13974.2
                                        No. Observations:                 5020
Date:                Tue, Aug 23 2022   Df Residuals:                     5019
Time:                        19:02:10   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.183e-02      3.939  8.187e-05 [2.342e-02,6.981e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0109  3.004e-03      3.640  2.724e-04 [5.048e-03,1.682e-02]
alpha[1]       0.0835  1.071e-02      7.794  6.476e-15   [6.249e-02,  0.104]
beta[1]        0.9089  1.148e-02     79.168      0.000     [  0.886,  0.931]
============================================================================

Covariance estimator: robust
"""

we see that this model took 13 iterations to converge. This is more than twice the number of computations it took for the simple arch to do the same, which was six. 

Now, since all the coefficients are significant and we get a higher log likelihood, this model instantly becomes our front runner for measuring volatility. But, lets examine more with higher order Lag GARCH.

Higher Lag GRCH 

It's been mathematically proven that no higher order GARCH models outperform the GARCH(1,1) when it comes to variance of market returns. This is due to the recursive nature in which the past conditional variances are computed, including one not only makes it redundant to include past squared residuals, since it already captures the effect, but also makes other conditional variances obsolete. From a mathematical point of view, all the effects of the conditional variants two days ago would be contained in the conditional variants yesterday, so there would be no need to include more than one GARCH component.

In [11]:
model_garch_1_2 = arch_model(df.returns[1:], mean='COnstant', vol='GARCH', p=1, q=2)
result_garch_1_2 = model_garch_1_2.fit() #param, update_freq = 5
result_garch_1_2.summary()

Iteration:      1,   Func. Count:      7,   Neg. LLF: 136466776694655.33
Iteration:      2,   Func. Count:     17,   Neg. LLF: 808459246.0256559
Iteration:      3,   Func. Count:     25,   Neg. LLF: 10137.459118305613
Iteration:      4,   Func. Count:     33,   Neg. LLF: 7008.430986114414
Iteration:      5,   Func. Count:     40,   Neg. LLF: 6974.173831538361
Iteration:      6,   Func. Count:     46,   Neg. LLF: 6971.511859365692
Iteration:      7,   Func. Count:     52,   Neg. LLF: 6970.616224326323
Iteration:      8,   Func. Count:     58,   Neg. LLF: 6973.892231638968
Iteration:      9,   Func. Count:     65,   Neg. LLF: 6970.063506713137
Iteration:     10,   Func. Count:     71,   Neg. LLF: 6970.058391826686
Iteration:     11,   Func. Count:     77,   Neg. LLF: 6970.058366757141
Iteration:     12,   Func. Count:     83,   Neg. LLF: 6970.05836622724
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6970.05836622724
            Iterations: 12
 

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13950.1
Method:            Maximum Likelihood   BIC:                           13982.7
                                        No. Observations:                 5020
Date:                Tue, Aug 23 2022   Df Residuals:                     5019
Time:                        19:15:22   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.184e-02      3.938  8.219e-05 [2.341e-02,6.982e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0109  2.908e-03      3.761  1.696e-04 [5.236e-03,1.663e-02]
alpha[1]       0.0835  1.189e-02      7.019  2.231e-12   [6.017e-02,  0.107]
beta[1]        0.9089      0.188      4.845  1.268e-06     [  0.541,  1.277]
beta[2]        0.0000      0.180      0.000      1.000     [ -0.352,  0.352]
============================================================================

Covariance estimator: robust
"""

We observe a p value of one for the beta two coefficient, this means we have a case of full multiocolinearity due to the relationship between conditional variances. Let see with GARCH 3.

In [12]:
model_garch_1_3 = arch_model(df.returns[1:], mean='COnstant', vol='GARCH', p=1, q=3)
result_garch_1_3 = model_garch_1_3.fit() #param, update_freq = 5
result_garch_1_3.summary()

Iteration:      1,   Func. Count:      8,   Neg. LLF: 48216.80281467884
Iteration:      2,   Func. Count:     20,   Neg. LLF: 30197.37469083751
Iteration:      3,   Func. Count:     31,   Neg. LLF: 645132346.733002
Iteration:      4,   Func. Count:     39,   Neg. LLF: 1579678655.9946866
Iteration:      5,   Func. Count:     47,   Neg. LLF: 7044.915118311412
Iteration:      6,   Func. Count:     55,   Neg. LLF: 7035.999098115932
Iteration:      7,   Func. Count:     63,   Neg. LLF: 6984.463705447777
Iteration:      8,   Func. Count:     71,   Neg. LLF: 6974.308102617722
Iteration:      9,   Func. Count:     79,   Neg. LLF: 7375.07612737505
Iteration:     10,   Func. Count:     88,   Neg. LLF: 6973.18032347493
Iteration:     11,   Func. Count:     96,   Neg. LLF: 6970.160511757096
Iteration:     12,   Func. Count:    103,   Neg. LLF: 6970.092279763916
Iteration:     13,   Func. Count:    110,   Neg. LLF: 6970.059721265443
Iteration:     14,   Func. Count:    117,   Neg. LLF: 6970.0586766

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13952.1
Method:            Maximum Likelihood   BIC:                           13991.2
                                        No. Observations:                 5020
Date:                Tue, Aug 23 2022   Df Residuals:                     5019
Time:                        19:17:39   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.179e-02      3.954  7.684e-05 [2.351e-02,6.972e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0109  8.158e-03      1.340      0.180 [-5.055e-03,2.693e-02]
alpha[1]       0.0835  6.061e-02      1.377      0.168   [-3.530e-02,  0.202]
beta[1]        0.9089      2.148      0.423      0.672      [ -3.302,  5.120]
beta[2]        0.0000      3.375      0.000      1.000      [ -6.616,  6.616]
beta[3]    3.7947e-13      1.294  2.932e-13      1.000      [ -2.536,  2.536]
=============================================================================

Covariance estimator: robust
"""

Here, we are bound to get the same exact same P value for both beta.

In [13]:
model_garch_2_1 = arch_model(df.returns[1:], mean='COnstant', vol='GARCH', p=2, q=1)
result_garch_2_1 = model_garch_2_1.fit() #param, update_freq = 5
result_garch_2_1.summary()

Iteration:      1,   Func. Count:      7,   Neg. LLF: 159537792891988.94
Iteration:      2,   Func. Count:     17,   Neg. LLF: 1848307647.4642267
Iteration:      3,   Func. Count:     25,   Neg. LLF: 10354.116095029069
Iteration:      4,   Func. Count:     33,   Neg. LLF: 7005.361877757425
Iteration:      5,   Func. Count:     40,   Neg. LLF: 8793.711867692436
Iteration:      6,   Func. Count:     47,   Neg. LLF: 7019.706996857094
Iteration:      7,   Func. Count:     54,   Neg. LLF: 6973.161801614009
Iteration:      8,   Func. Count:     61,   Neg. LLF: 7010.720504720297
Iteration:      9,   Func. Count:     69,   Neg. LLF: 6967.937761572108
Iteration:     10,   Func. Count:     76,   Neg. LLF: 6967.731247505904
Iteration:     11,   Func. Count:     82,   Neg. LLF: 6967.731020076308
Iteration:     12,   Func. Count:     87,   Neg. LLF: 6967.7310200762295
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6967.731020076308
            Iterations: 

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6967.73
Distribution:                  Normal   AIC:                           13945.5
Method:            Maximum Likelihood   BIC:                           13978.1
                                        No. Observations:                 5020
Date:                Tue, Aug 23 2022   Df Residuals:                     5019
Time:                        19:19:56   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.187e-02      3.922  8.780e-05 [2.329e-02,6.982e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0129  4.097e-03      3.158  1.589e-03  [4.908e-03,2.097e-02]
alpha[1]       0.0547  1.665e-02      3.286  1.017e-03  [2.208e-02,8.735e-02]
alpha[2]       0.0389  2.345e-02      1.659  9.709e-02 [-7.056e-03,8.488e-02]
beta[1]        0.8974  1.712e-02     52.415      0.000      [  0.864,  0.931]
=============================================================================

Covariance estimator: robust
"""

we can still see the additional coefficient is not significantly different from zero at the five percent significance level.
This experiment gave us further proof that the GARCH(1,1) is the best model for measuring volatility of returns, and there's no need to rely on overly complicated models.